### Import dataset

In [1]:
import pandas as pd
data = pd.read_csv("data/dataset.csv")
data

,Sentimen,Komen
0,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...
395,positive,Bangga sama suami yg selalu ingat istri disela...
396,positive,Apaoun pekerjaannya yg penting halal u tuk men...
397,positive,Gojek itu mayoritas pegangguran yang lama gak ...
398,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...


### Case Folding dan Pembersihan Teks
Mengubah teks menjadi bentuk yang seragam, biasanya mengubah menjadi text lowercase.
Membersihkan teks dari simbol simbol yang tidak diperlukan.

In [2]:
import re 
def casefolding(komen):
    komen = komen.lower()
    komen = komen.strip(" ")
    komen = re.sub(r'(<[A-Za-z0-9]+>)|(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)',"",komen)
    return komen
data['Komen'] = data['Komen'].apply(casefolding)
data.head()

,Sentimen,Komen
0,negative,tolol gak ada hubungan nya keguguran dgn pake...
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,kmrn termewek2 skr lengket lg duhhh kok labil ...
3,negative,intinya kalau kesel dengan att nya gausah ke a...
4,negative,hadewwwww permpuan itu lgsakit jiwaknp harus d...


### Tokenization
Memecah sebuah teks atau kalimat menjadi bagian-bagian yang lebih kecil

In [3]:
def token(komen):
    nstr = komen.split(" ")
    dat = []
    a = -1

    for i in nstr:
        a = a + 1

    if i == "":
        dat.append(a)

    p = 0
    b = 0

    for q in dat:
        b = q - p
        del nstr[b]
        p = p + 1

    return nstr

data["Komen"] = data["Komen"].apply(token)
data.head(100)

,Sentimen,Komen
0,negative,"[, tolol, gak, ada, hubungan, nya, keguguran, ..."
1,negative,"[geblek, lo, tatacowo, bgt, dibela2in, balikan..."
2,negative,"[kmrn, termewek2, skr, lengket, lg, duhhh, kok..."
3,negative,"[intinya, kalau, kesel, dengan, att, nya, gaus..."
4,negative,"[hadewwwww, permpuan, itu, lgsakit, jiwaknp, h..."
...,...,...
95,negative,"[yg, nama, ny, plagiat, ya, plagiat, walaupun,..."
96,negative,"[dasar, plagiat, semuanya, ga, bisa, kreatipme..."
97,negative,"[dan, dengan, begonya, dia, bilang, apanya, ya..."
98,negative,"[itu, pacarnya, anggel, pietrs, kali, dan, cow..."


### Filtering
Menghilangkan kata yang kurang bermakna, seperti kata penghubung atau yang lainya

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def stopword_removal(komen):
    filtering = stopwords.words("indonesian")
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
        
    fit = filter(myFunc, komen)
    for x in fit:
        data.append(x)

    return data

data["Komen"] = data["Komen"].apply(stopword_removal)
data.head(100)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


,Sentimen,Komen
0,negative,"[, tolol, gak, hubungan, nya, keguguran, dgn, ..."
1,negative,"[geblek, lo, tatacowo, bgt, dibela2in, balikan..."
2,negative,"[kmrn, termewek2, skr, lengket, lg, duhhh, lab..."
3,negative,"[intinya, kesel, att, nya, gausah, anaknya, ka..."
4,negative,"[hadewwwww, permpuan, lgsakit, jiwaknp, yg, jd..."
...,...,...
95,negative,"[yg, nama, ny, plagiat, ya, plagiat, laku, tp,..."
96,negative,"[dasar, plagiat, ga, kreatipmemalukab, bgt]"
97,negative,"[begonya, bilang, apanya, yaaa, liriknya, aja,..."
98,negative,"[pacarnya, anggel, pietrs, kali, cowok, bocah,..."


### Stemming
Mengubah kata menjadi kata dasar atau menghilangkan imbuhan yang terdapat pada suatu kata

In [5]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(komen):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kt = []

    for w in komen:
        dt = stemmer.stem(w)
        kt.append(dt)

    dt_clean = []
    dt_clean = " ".join(kt)
    return dt_clean

data["Komen"] = data["Komen"].apply(stemming)

data.to_csv("data/data_clean.csv", index=False)
data_clean = pd.read_csv("data/data_clean.csv")
data_clean.head()

,Sentimen,Komen
0,negative,tolol gak hubung nya gugur dgn pake hijab sya...
1,negative,geblek lo tatacowo bgt dibela2in balikanhadeww...
2,negative,kmrn termewek2 skr lengket lg duhhh labil bgt ...
3,negative,inti kesel att nya gausah anak kasi kembang ps...
4,negative,hadewwwww permpuan lgsakit jiwaknp yg jd peran...


### Mengkategorikan data menjaadi string dan kategori

In [6]:
data_clean = data_clean.astype({'Komen' : 'string'})
data_clean = data_clean.astype({'Sentimen' : 'category'})
data_clean.dtypes

Sentimen          category
Komen       string[python]
dtype: object

### TF-IDF ( Term Frequency-Inverse Document Frequency)
Mengukur seberapa sering sebuah kata muncul dalam sebuah dataset atau dokumen

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdf = TfidfVectorizer()
data_tfIdf = tfIdf.fit_transform(data_clean["Komen"].astype("U"))
print(data_tfIdf.shape)

(400, 2669)


### Splitting Data
membagi dataset menjadi subset yang berbeda untuk pelatihan (training) dan pengujian (testing)

In [8]:
from sklearn.model_selection import train_test_split

x = data_tfIdf
y = data_clean["Sentimen"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("X_train : ", x_train.shape)
print("X_test : ", x_test.shape)
print("y_train : ", y_train.shape)
print("y_test : ", y_test.shape)

X_train :  (320, 2669)
X_test :  (80, 2669)
y_train :  (320,)
y_test :  (80,)


### Implementasi Multinomial Naive Bayes

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(x_train, y_train)
predicted = clf.predict(x_test)
print("MultinomialNB Accuracy : ", accuracy_score(y_test,predicted))
print("MultinomialNB Preccision : ", precision_score(y_test, predicted, average="binary", pos_label="negative"))
print("MultinomialNB Recall : ", recall_score(y_test, predicted, average="binary", pos_label="negative"))
print("MultinomialNB F1 Score : ", f1_score(y_test, predicted, average="binary", pos_label="negative"))
print(f"Confusion Matrix : \n {confusion_matrix(y_test, predicted)}")
print("======================================================")
print(classification_report(y_test, predicted, zero_division=0))

MultinomialNB Accuracy :  0.9125
MultinomialNB Preccision :  0.9743589743589743
MultinomialNB Recall :  0.8636363636363636
MultinomialNB F1 Score :  0.9156626506024097
Confusion Matrix : 
 [[38  6]
 [ 1 35]]
              precision    recall  f1-score   support

    negative       0.97      0.86      0.92        44
    positive       0.85      0.97      0.91        36

    accuracy                           0.91        80
   macro avg       0.91      0.92      0.91        80
weighted avg       0.92      0.91      0.91        80



### Mencoba Prediksi

In [16]:
# Mencoba prediksi manual dari list

dt_pred = ["jahat banget kamu"]
dt_pred_tfid = tfIdf.transform(dt_pred)

my_pred = clf.predict(dt_pred_tfid)
print(my_pred[0])

negative


Coba dengan dataset

In [11]:
# mencoba prediksi dataset
data_predict = pd.read_csv("data_prediksi/data_prediksi_clean.csv")
data_predict.head()

,Komen
0,jln jatibarupolisi tdk bs gertak gubernur eman...
1,cewe lho kayak rasain sibuk jaga rasain sakit ...
2,kepingin gudeg mbarek bu hj amad foto google s...
3,jln jatibarubagian wilayah tn abangpengaturan ...
4,sharing alam aja kemarin jam 1800 batalin tike...


In [12]:
tfid_data_predict = tfIdf.transform(data_predict["Komen"])
my_predict = clf.predict(tfid_data_predict)
my_predict

array(['positive', 'negative', 'positive', 'negative', 'positive',
       'negative', 'negative', 'negative', 'negative', 'positive',
       'positive', 'negative', 'positive', 'positive', 'negative',
       'negative', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'positive', 'negative', 'positive',
       'positive', 'negative', 'negative', 'positive', 'positive',
       'negative', 'positive', 'negative', 'positive', 'negative',
       'positive', 'negative', 'negative', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'negative', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'negative', 'positive', 'negative', 'positive', 'negative',
       'negative', 'positive', 'positive', 'positive', 'positi

In [13]:
data_predict["Sentimen"] = my_predict
data_predict.to_csv("data_prediksi/hasil_prediksi.csv", index=False)

hasil_predict = pd.read_csv("data_prediksi/hasil_prediksi.csv")
hasil_predict

,Komen,Sentimen
0,jln jatibarupolisi tdk bs gertak gubernur eman...,positive
1,cewe lho kayak rasain sibuk jaga rasain sakit ...,negative
2,kepingin gudeg mbarek bu hj amad foto google s...,positive
3,jln jatibarubagian wilayah tn abangpengaturan ...,negative
4,sharing alam aja kemarin jam 1800 batalin tike...,positive
...,...,...
94,bodoh mu bong suruh baca baca pbb cikal masyum...,negative
95,mudah2an terupload smua z mudik karna fans mod...,positive
96,orang dukung khilafah black mail,negative
97,sok akrab ye mention mention gue maling aje y...,negative


#### Membuat Model 

In [14]:
from sklearn.linear_model import LogisticRegression
import pickle

model = LogisticRegression()
model.fit(x_train, y_train)

pickle.dump(tfIdf, open("pickle/tfIdf.pkl", mode="wb"))
pickle.dump(clf, open("pickle/clf.pkl", mode="wb"))
pickle.dump(model, open("pickle/clf.pkl", mode="wb"))

In [17]:
with open('pickle/tfIdf.pkl', mode='rb') as tf:
    tfid_load = pickle.load(tf)
with open('pickle/model.pkl', mode='rb') as model:
    model_load = pickle.load(model)
with open('pickle/clf.pkl', mode='rb') as cl:
    clf_load = pickle.load(cl)

dt_pred = ["baik banget kamu"]
dt_pred_tfid = tfid_load.transform(dt_pred)
pred = clf_load.predict(dt_pred_tfid)

print(pred[0])

positive
